In [ ]:
from sklearn.datasets import load_boston
boston = load_boston()

In [ ]:
print(boston.keys())

In [ ]:
print(boston.data.shape)

In [ ]:
print(boston.feature_names)

In [ ]:
print(boston.DESCR)

In [ ]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

In [ ]:

data = pd.DataFrame(boston.data)
data.columns = boston.feature_names

In [ ]:
X, y = data.iloc[:,:-1],data.iloc[:,-1]

In [ ]:
data_dmatrix = xgb.DMatrix(data=X,label=y)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)


In [ ]:
# TEsting sklearn example
from sklearn.datasets import load_diabetes
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
X_d, y_d = load_diabetes(return_X_y=True)
regressor =  DecisionTreeRegressor(random_state=0)
cross_val_score(regressor, X_d, y_d, cv=10)

In [ ]:
# Lets load use sklearn DTR with boston
b_regressor = DecisionTreeRegressor(random_state=1)
cross_val_score(b_regressor, X_train, y_train, cv = 10)

In [ ]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

In [ ]:
%time xg_reg.fit(X_train,y_train)
%time b_regressor.fit(X_train,y_train)
%time b_preds = b_regressor.predict(X_test)
%time preds = xg_reg.predict(X_test)
#b_preds, preds

In [ ]:
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))
rmse = np.sqrt(mean_squared_error(y_test, b_preds))
print("basic RMSE: %f" % (rmse))

In [ ]:
rmse = np.sqrt(mean_squared_error(y_test, (preds + b_preds)/2 ))
print("RMSE: %f" % (rmse)) 

In [ ]:
for i in range(len(preds)):
    print("Y=", y_test.iloc[i], " preds=",preds[i],", b_preds=",b_preds[i],", mean_preds=",(preds[i] + b_preds[i])/2)

In [ ]:
params = {"objective":"reg:linear",'colsample_bytree': 0.3,'learning_rate': 0.1,
                'max_depth': 5, 'alpha': 10}

cv_results = xgb.cv(dtrain=data_dmatrix, params=params, nfold=3,
                    num_boost_round=50,early_stopping_rounds=10,metrics="rmse", as_pandas=True, seed=123)


In [ ]:
cv_results.head, cv_results.shape

In [ ]:
print((cv_results["test-rmse-mean"]).tail(1))

In [ ]:
xg_reg = xgb.train(params=params, dtrain=data_dmatrix, num_boost_round=10)

In [ ]:
import matplotlib.pyplot as plt

xgb.plot_tree(xg_reg,num_trees=0)
plt.rcParams['figure.figsize'] = [50, 20]
plt.show()

In [ ]:
xgb.plot_importance(xg_reg)
plt.rcParams['figure.figsize'] = [50, 50]
plt.show()